In [1]:
import os
import sys
import re
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

import api


/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meta_df = pd.read_csv("../../intermediate_data/cpc/metadata.csv")

In [3]:
meta_df['agenda_pages'] = (meta_df['filename']=='agenda')*(meta_df['pages'])
meta_df['supplemental_pages'] = (meta_df['filename']=='supplemental-docs')*(meta_df['pages'])
meta_df['minutes_pages'] = (meta_df['filename']=='minutes')*(meta_df['pages'])
meta_df2 = meta_df.groupby(['year','date']).agg(
    agenda_pages = ('agenda_pages','sum'),
    supplemental_pages = ('supplemental_pages','sum'),
    minutes_pages = ('minutes_pages','sum')
).reset_index()
meta_df2['has_all'] = (meta_df2['agenda_pages']>0) & (meta_df2['supplemental_pages']>0) & (meta_df2['minutes_pages']>0)
meta_df2['total_pages'] = meta_df2['agenda_pages'] + meta_df2['supplemental_pages'] + meta_df2['minutes_pages']

In [4]:
idx = meta_df2['has_all']
meta_df2.loc[idx].to_csv("../../intermediate_data/cpc/meetings_metadata.csv", index=False, header=True)

In [5]:
meta_df2.loc[idx]

,year,date,agenda_pages,supplemental_pages,minutes_pages,has_all,total_pages
392,2018,2018-05-10,8,219,10,True,237
393,2018,2018-05-23,9,31,10,True,50
394,2018,2018-06-14,9,81,10,True,100
396,2018,2018-07-12,7,75,9,True,91
397,2018,2018-07-26,5,49,5,True,59
...,...,...,...,...,...,...,...
556,2024,2024-07-25,8,33,8,True,49
557,2024,2024-08-08,11,10,11,True,32
559,2024,2024-09-12,12,65,10,True,87
560,2024,2024-09-26,6,1348,6,True,1360
